### Title: 09_logistic_regression_odds_ratios
### Purpose: Calculate Odds Ratios for specific ingredients with selected nodes from TaxaHFE
### Date: April 15, 2024
### Author: Jules Larke

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 

In [2]:
otu = pd.read_csv('../../data/03/ingred_otu_e_adjust_drywt_no_salt_water.txt', sep='\t')
cov = pd.read_csv('/Users/jules.larke/work/project/ingredient_tree/data/00/wweia_covariates.csv')

y_train = pd.read_csv('/Users/jules.larke/work/project/ingredient_tree/data/04/processed_for_ml/binary_class/y_train_class.csv')
y_test = pd.read_csv('/Users/jules.larke/work/project/ingredient_tree/data/04/processed_for_ml/binary_class/y_test_class.csv')

In [3]:
outcome = pd.concat([y_test, y_train])

In [4]:
outcome = outcome.merge(cov, on='SEQN')

In [5]:
outcome['SEQN'] = outcome['SEQN'].astype(str)

In [6]:
outcome = pd.get_dummies(outcome, columns=['Sex', 'education', 'Ethnicity', 'ever_smoker', 'diabetes', 'hypertension'], drop_first=True, dtype=np.float64) 

In [7]:
pid = pd.concat([y_train['SEQN'], y_test['SEQN']])

In [8]:
pid = pid.astype(str)

In [9]:
otu_filter = otu[otu.columns[otu.columns.isin(pid)]]

In [10]:
otu_filter.shape

(566, 12909)

In [11]:
otu_filter = pd.concat([otu['clade_name'], otu['Main.food.description'], otu_filter],axis=1)

In [12]:
otu_filter['clade_name'] = otu_filter['clade_name'].str.split('|', expand=True)[1] # L2

In [13]:
nonalc = otu_filter[otu_filter['clade_name'].str.contains('L2_Nonalcoholic_beverages', case=False)]

In [14]:
nonalc = nonalc.drop(columns='clade_name')

In [15]:
nonalc = nonalc.transpose()

In [16]:
new_header = nonalc.iloc[0] #grab the first row for the header
nonalc = nonalc[1:] #take the data less the header row
nonalc.columns = new_header #set the header row as the df header

In [17]:
nonalc = nonalc.reset_index()

In [18]:
nonalc = nonalc.rename(columns={'index':'SEQN'})

In [19]:
nonalc['SEQN'] = nonalc['SEQN'].astype(str)

In [20]:
nonalc = nonalc.merge(outcome, on='SEQN')

In [21]:
nonalc['crp_class'] = nonalc.pop('crp_class')

In [22]:
nonalc = nonalc.drop(columns=['education_unknown'])

In [23]:
#nonalc = nonalc.drop(columns=['Sex_Male', 'education_some college', 'Ethnicity_Other_Multi-Racial', 'ever_smoker_yes', 'diabetes_yes', 'hypertension_yes'])

In [24]:
nonalc.iloc[:,1:35]

,Beverages chocolate powder no sugar added,Strawberryflavor beverage mix powder,Orangeflavor drink breakfast type powder,Coffee brewed prepared with tap water,Beverages coffee instant regular half the caffeine,Beverages coffee brewed espresso restaurantprepared,Beverages coffee brewed prepared with tap water decaffeinated,Beverages coffee ready to drink milk based sweetened,Beverages coffee substitute cereal grain beverage powder,Beverages tea Oolong brewed,...,education_high school graduate or equivalent,education_less than high school graduate,education_some college,Ethnicity_Non-Hispanic_Black,Ethnicity_Non-Hispanic_White,Ethnicity_Other_Hispanic,Ethnicity_Other_Multi-Racial,ever_smoker_yes,diabetes_yes,hypertension_yes
0,0.0,0.0,0.0,0.0,0.0,0.0,1.311951,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3.255134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.372847,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.494963,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.064302,0.0,0.0,0.0,0.0,0.0,0.193316,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12904,0.300717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [25]:
nonalc_x = nonalc.iloc[:,1:35]

In [26]:
nonalc_y = nonalc.iloc[:,35:]

In [27]:
nonalc_x_train, nonalc_x_test, nonalc_y_train, nonalc_y_test = train_test_split(nonalc_x, nonalc_y, stratify=nonalc_y['crp_class'], shuffle=True, test_size=0.2)

In [28]:
# convert to proper format, need to cast to float for statmodels (can throw error)
nonalc_x_train = nonalc_x_train.values.reshape(-1, 34).astype(np.float64)
nonalc_y_train = nonalc_y_train.values.reshape(-1, 1).astype(np.float64)

In [29]:
nonalc_x_test = nonalc_x_test.values.reshape(-1, 34).astype(np.float64)
nonalc_y_test = nonalc_y_test.values.reshape(-1, 1).astype(np.float64)

In [82]:
# logistic regression model for nonalc bev
nonalc_x_train_ = sm.add_constant(nonalc_x_train)
nonalc_sm = sm.Logit(nonalc_y_train, nonalc_x_train_).fit(method="ncg")
print(nonalc_sm.summary())

# coefficients are in log-odds terms
nonalc_x_test_ = sm.add_constant(nonalc_x_test)
nonalc_y_pred = nonalc_sm.predict(exog=nonalc_x_test_)

# converting probability to labels
def convert_prob_to_label(prob, cutoff = 0.5):
    label = None
    if prob > cutoff:
        label = 1
    else:
        label = 0
    return label

nonalc_pred_labels = list(map(convert_prob_to_label, nonalc_y_pred))
nonalc_pred_labels = np.asarray(nonalc_pred_labels)

Optimization terminated successfully.
         Current function value: 0.514955
         Iterations: 18
         Function evaluations: 20
         Gradient evaluations: 20
         Hessian evaluations: 18
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10327
Model:                          Logit   Df Residuals:                    10292
Method:                           MLE   Df Model:                           34
Date:                Thu, 04 Apr 2024   Pseudo R-squ.:                  0.2570
Time:                        17:58:18   Log-Likelihood:                -5317.9
converged:                       True   LL-Null:                       -7157.3
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const

In [83]:
nonalc_marginal = nonalc_sm.get_margeff(at='overall')

In [84]:
nonalc_summary = nonalc_marginal.summary_frame()

In [85]:
nonalc_summary.index = nonalc.columns[1:35]

In [86]:
nonalc_summary.to_csv('../../data/09/nonalc_bev_odds_all_ingredients_all_cov.csv',index=True)

In [87]:
# compute accuracy for the model on the held out test data
balanced_accuracy_score(nonalc_y_test, nonalc_pred_labels)

0.7595688374814348

### L2_pastas_cooked_cereals

In [88]:
pcc = otu_filter[otu_filter['clade_name'].str.contains('L2_pastas_cooked_cereals', case=False)]

In [89]:
pcc = pcc.drop(columns='clade_name')

In [90]:
pcc = pcc.transpose()

In [91]:
new_header = pcc.iloc[0] #grab the first row for the header
pcc = pcc[1:] #take the data less the header row
pcc.columns = new_header #set the header row as the df header

In [92]:
pcc = pcc.reset_index()

In [93]:
pcc = pcc.rename(columns={'index':'SEQN'})

In [94]:
pcc['SEQN'] = pcc['SEQN'].astype(str)

In [95]:
pcc = pcc.merge(outcome, on='SEQN')

In [96]:
pcc['crp_class'] = pcc.pop('crp_class')

In [99]:
pcc = pcc.drop(columns=['education_unknown'])

In [103]:
pcc.iloc[:,1:32]

,Pasta dry enriched,Macaroni vegetable enriched cooked,Noodles egg dry enriched,Noodles egg enriched cooked,Noodles egg spinach enriched cooked,Noodles chinese chow mein,Pasta wholewheat cooked,Rice white longgrain regular raw enriched,Rice brown longgrain raw,Wild rice raw,...,education_high school graduate or equivalent,education_less than high school graduate,education_some college,Ethnicity_Non-Hispanic_Black,Ethnicity_Non-Hispanic_White,Ethnicity_Other_Hispanic,Ethnicity_Other_Multi-Racial,ever_smoker_yes,diabetes_yes,hypertension_yes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.531149,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.896633,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.380216,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [104]:
pcc_x = pcc.iloc[:,1:32]

In [105]:
pcc_y = pcc.iloc[:,32:]

In [106]:
pcc_x_train, pcc_x_test, pcc_y_train, pcc_y_test = train_test_split(pcc_x, pcc_y, stratify=pcc_y['crp_class'], shuffle=True, test_size=0.2)

In [107]:
# convert to proper format, need to cast to float for statmodels (can throw error)
pcc_x_train = pcc_x_train.values.reshape(-1, 31).astype(np.float64)
pcc_y_train = pcc_y_train.values.reshape(-1, 1).astype(np.float64)

In [108]:
pcc_x_test = pcc_x_test.values.reshape(-1, 31).astype(np.float64)
pcc_y_test = pcc_y_test.values.reshape(-1, 1).astype(np.float64)

In [109]:
# logistic regression model for pcc bev
pcc_x_train_ = sm.add_constant(pcc_x_train)
pcc_sm = sm.Logit(pcc_y_train, pcc_x_train_).fit(method="ncg")
print(pcc_sm.summary())

# coefficients are in log-odds terms
pcc_x_test_ = sm.add_constant(pcc_x_test)
pcc_y_pred = pcc_sm.predict(exog=pcc_x_test_)

# converting probability to labels
def convert_prob_to_label(prob, cutoff = 0.5):
    label = None
    if prob > cutoff:
        label = 1
    else:
        label = 0
    return label

pcc_pred_labels = list(map(convert_prob_to_label, pcc_y_pred))
pcc_pred_labels = np.asarray(pcc_pred_labels)

Optimization terminated successfully.
         Current function value: 0.512193
         Iterations: 18
         Function evaluations: 20
         Gradient evaluations: 20
         Hessian evaluations: 18
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10327
Model:                          Logit   Df Residuals:                    10295
Method:                           MLE   Df Model:                           31
Date:                Thu, 04 Apr 2024   Pseudo R-squ.:                  0.2610
Time:                        18:03:50   Log-Likelihood:                -5289.4
converged:                       True   LL-Null:                       -7157.3
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const

In [110]:
pcc_marginal = pcc_sm.get_margeff(at='overall')

In [111]:
pcc_summary = pcc_marginal.summary_frame()

In [112]:
pcc_summary.index

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
       'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21',
       'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x31'],
      dtype='object')

In [113]:
pcc_summary.index = pcc.columns[1:32]

In [114]:
pcc_summary.to_csv('../../data/09/pcc_odds_all_ingredients_all_cov.csv',index=True)

In [115]:
# compute accuracy for the model on the held out test data
balanced_accuracy_score(pcc_y_test, pcc_pred_labels)

0.742573173110101

## L3_milk_cow

In [116]:
otu_filter = otu[otu.columns[otu.columns.isin(pid)]]

In [117]:
otu_filter = pd.concat([otu['clade_name'], otu['Main.food.description'], otu_filter],axis=1)

In [118]:
otu_filter['clade_name'] = otu_filter['clade_name'].str.split('|', expand=True)[2] # L3

In [119]:
milk = otu_filter[otu_filter['clade_name'].str.contains('L3_milk_cow', case=False)]

In [120]:
milk = milk.drop(columns='clade_name')

In [121]:
milk = milk.transpose()

In [122]:
new_header = milk.iloc[0] #grab the first row for the header
milk = milk[1:] #take the data less the header row
milk.columns = new_header #set the header row as the df header

In [123]:
milk = milk.reset_index()

In [124]:
milk = milk.rename(columns={'index':'SEQN'})

In [125]:
milk['SEQN'] = milk['SEQN'].astype(str)

In [126]:
milk = milk.merge(outcome, on='SEQN')

In [127]:
milk = milk.drop(columns=['education_unknown'])

In [128]:
milk['crp_class'] = milk.pop('crp_class')

In [131]:
milk.iloc[:,1:23]

,Milk whole 325 milkfat with added vitamin D,Milk reduced fat fluid 2 milkfat with added vitamin A and vitamin D,Milk lowfat fluid 1 milkfat with added vitamin A and vitamin D,Milk nonfat fluid with added vitamin A and vitamin D fat free or skim,Milk dry nonfat regular without added vitamin A and vitamin D,Milk averaged fat with added vitamin A and D,Milk buttermilk fluid cultured lowfat,Milk chocolate fat free with added vitamin A and vitamin D,Age,family_pir,...,education_high school graduate or equivalent,education_less than high school graduate,education_some college,Ethnicity_Non-Hispanic_Black,Ethnicity_Non-Hispanic_White,Ethnicity_Other_Hispanic,Ethnicity_Other_Multi-Racial,ever_smoker_yes,diabetes_yes,hypertension_yes
0,1.146168,0.0,0.0,0.0,0.0,0.003927,0.0,0.0,44.0,5.397605e-79,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,5.381802,0.0,0.0,0.0,0.0,0.181459,0.0,0.0,76.0,1.720000e+00,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,10.084237,0.0,0.0,0.0,0.0,2.105529,0.0,0.0,24.0,5.000000e+00,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,3.063143,0.0,0.0,0.0,0.0,0.514372,0.0,0.0,18.0,6.800000e-01,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,89.510754,0.0,0.0,0.0,0.0,0.043301,0.0,0.0,44.0,2.830000e+00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,7.500000e-01,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
12905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,2.060000e+00,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
12906,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,2.140000e+00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,1.680000e+00,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [132]:
milk_x = milk.iloc[:,1:23]

In [133]:
milk_y = milk.iloc[:,23:]

In [134]:
milk_x_train, milk_x_test, milk_y_train, milk_y_test = train_test_split(milk_x, milk_y, stratify=milk_y['crp_class'], shuffle=True, test_size=0.2)

In [135]:
# convert to proper format, need to cast to float for statmodels (can throw error)
milk_x_train = milk_x_train.values.reshape(-1, 22).astype(np.float64)
milk_y_train = milk_y_train.values.reshape(-1, 1).astype(np.float64)

In [136]:
milk_x_test = milk_x_test.values.reshape(-1, 22).astype(np.float64)
milk_y_test = milk_y_test.values.reshape(-1, 1).astype(np.float64)

In [137]:
# logistic regression model for milk bev
milk_x_train_ = sm.add_constant(milk_x_train)
milk_sm = sm.Logit(milk_y_train, milk_x_train_).fit(method="ncg")
print(milk_sm.summary())

# coefficients are in log-odds terms
milk_x_test_ = sm.add_constant(milk_x_test)
milk_y_pred = milk_sm.predict(exog=milk_x_test_)

# converting probability to labels
def convert_prob_to_label(prob, cutoff = 0.5):
    label = None
    if prob > cutoff:
        label = 1
    else:
        label = 0
    return label

milk_pred_labels = list(map(convert_prob_to_label, milk_y_pred))
milk_pred_labels = np.asarray(milk_pred_labels)

Optimization terminated successfully.
         Current function value: 0.513070
         Iterations: 13
         Function evaluations: 14
         Gradient evaluations: 14
         Hessian evaluations: 13
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10327
Model:                          Logit   Df Residuals:                    10304
Method:                           MLE   Df Model:                           22
Date:                Thu, 04 Apr 2024   Pseudo R-squ.:                  0.2597
Time:                        18:06:00   Log-Likelihood:                -5298.5
converged:                       True   LL-Null:                       -7157.3
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const

In [138]:
milk_marginal = milk_sm.get_margeff(at='overall')

In [139]:
milk_summary = milk_marginal.summary_frame()

In [140]:
milk_summary.index

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11',
       'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21',
       'x22'],
      dtype='object')

In [141]:
milk_summary.index = milk.columns[1:23]

In [142]:
milk_summary.to_csv('../../data/09/milk_odds_all_ingredients_all_cov.csv',index=True)

In [143]:
# compute accuracy for the model on the held out test data
balanced_accuracy_score(milk_y_test, milk_pred_labels)

0.754586014972171